In [15]:
import importlib
from pathlib import Path
from pprint import pprint
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


In [16]:
from bsm_multi_agents.graph import agent_graph
importlib.reload(agent_graph)
from bsm_multi_agents.graph.agent_graph import build_app, WorkflowState

In [17]:
# app = build_app()
# csv_path = Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv"
# init_state: WorkflowState = {
#     "csv_file_path": str(csv_path),
#     "messages": [HumanMessage(content=f"Load CSV from: {csv_path}")],
# }

# final_state = app.invoke(
#     init_state,
#     config={"configurable": {"thread_id": "run-1"}}
# )

In [18]:
"""Run the complete workflow."""

# Build the application graph
print("=" * 80)
print("Building Agent Graph...")
print("=" * 80)
app = build_app()
csv_path = Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv"
init_state: WorkflowState = {
    "csv_file_path": str(csv_path),
    "messages": [HumanMessage(content=f"Load CSV from: {csv_path}")],
}
# Run the workflow
print("=" * 80)
print("Starting Workflow Execution...")
print("=" * 80)
print("\nWorkflow stages:")
print("  1. data_loader → Load CSV data")
print("  2. calculator → Calculate BSM prices and Greeks")
print("  3. validator → Validate results")
print("  4. summary_generator → Generate markdown summary")
print("  5. chart_generator → Create visualization charts")
print("  6. report_generator → Generate Word report")
print("\n" + "=" * 80 + "\n")

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

# Print results
print("\n" + "=" * 80)
print("Workflow Completed Successfully!")
print("=" * 80)

print(f"\n📊 Final State Summary:")
print(f"  State keys: {list(final_state.keys())}")

print(f"\n✅ Workflow Progress:")
stages = [
    ("csv_data", "CSV Data Loaded"),
    ("bsm_results", "BSM Results Calculated"),
    ("greeks_results", "Greeks Calculated"),
    ("validate_results", "Results Validated"),
    ("report_md", "Summary Generated"),
    ("chart_results", "Charts Created"),
    ("report_path", "Word Report Generated"),
]

for key, description in stages:
    status = "✅" if key in final_state else "❌"
    print(f"  {status} {description}")
    if key in final_state and key == "csv_data":
        print(f"      → {len(final_state[key])} rows loaded")
    elif key in final_state and key == "chart_results":
        print(f"      → {len(final_state[key])} charts created")
    elif key in final_state and key == "report_path":
        print(f"      → {final_state[key]}")

# Print errors if any
if "errors" in final_state and final_state["errors"]:
    print(f"\n⚠️  Errors encountered:")
    for error in final_state["errors"]:
        print(f"  - {error}")

# Print final report path
if "report_path" in final_state:
    print(f"\n" + "=" * 80)
    print(f"📄 Final Report Generated:")
    print(f"   {final_state['report_path']}")
    print("=" * 80)
else:
    print(f"\n⚠️  Warning: Final report was not generated")

Building Agent Graph...
Starting Workflow Execution...

Workflow stages:
  1. data_loader → Load CSV data
  2. calculator → Calculate BSM prices and Greeks
  3. validator → Validate results
  4. summary_generator → Generate markdown summary
  5. chart_generator → Create visualization charts
  6. report_generator → Generate Word report


[data_loader] LLM: ChatOllama (qwen3:8b), Tools: ['csv_loader']
[Router] bsm_results or greeks_results missing → routing to calculator
[calculator] LLM: ChatOllama (qwen3:8b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[Router] validate_results missing → routing to validator
[validator] LLM: ChatOllama (qwen3:8b), Tools: ['batch_greeks_validator']
[Router] report_md missing → routing to summary_generator


ValidationError: 1 validation error for generate_summary
state
  Field required [type=missing, input_value={'validate_results': '"[{...te.md', 'save_md': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [ ]:
def print_resp(resp):
    step_num = 1
    for message in resp["messages"]:
        if isinstance(message, HumanMessage):
            print(f"Step {step_num} - inputs:")
            print(f"   {message.content[:200]}..." if len(message.content) > 200 else f"   {message.content}")
            print()
            step_num += 1

        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                # Agent 决定调用工具
                print(f"Step {step_num} - Agent decide tools used:")
                for tool_call in message.tool_calls:
                    print(f"   Tool name: {tool_call['name']}")
                    print(f"   Tool parameters: {tool_call['args']}")
                print()
                step_num += 1
            elif message.content:
                print(f"Step {step_num} - Agent outputs:")
                print(f"   {message.content}")
                print()
                step_num += 1

        elif isinstance(message, ToolMessage):
            print(f"Step {step_num} - outputs:")
            print(f"   Tool name: {message.name}")
            # result_preview = message.content[:300] + "..." if len(message.content) > 300 else message.content
            result_preview = message.content
            print(f"   Outputs: {result_preview}")
            print()
            step_num += 1

    print(f"\n{'='*80}")
    print(f"Final outputs:")
    print(f"{'='*80}\n")
    print(resp["messages"][-1].content)
